In [47]:
import os
import requests
import queue
import pandas as pd
import threading
from bs4 import BeautifulSoup
from tqdm import tqdm

DATA_PATH = "../data"
DICTS_PATH = "../dicts"
DATA_FILENAME = "valid.csv"
INFO_FILE = "info.csv"

data_file = os.path.join(DATA_PATH, DATA_FILENAME)
info_file = os.path.join(DATA_PATH, INFO_FILE)

In [48]:
valid_df = pd.read_csv(data_file, index_col=0, dtype={"purch": str, "winer": str})
info_df = pd.read_csv(info_file, dtype={"winer": str})
info_df = info_df.set_index("winer")

In [49]:
info_df["phone_number"] = ""
info_df["address"] = ""
info_df["email"] = ""
info_df["company_name"] = ""

In [50]:
headers = {'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
           'sec-ch-ua-mobile':'0',
           'Upgrade-Insecure-Requests': '1',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
                    }

def get_winer_info_vie_purch(purch, q):   
    url = "https://zakupki.gov.ru/epz/order/notice/ea44/view/supplier-results.html?regNumber=" + str(purch) 
    r = requests.get(url, headers = headers)
    text = r.content.decode('utf-8')
    soup = BeautifulSoup(text)
    winer_list = soup.find_all('table', {'class': 'blockInfo__table tableBlock'})
    if len(winer_list) > 3:
        soup = BeautifulSoup(str(winer_list[3]))
    elif len(winer_list) == 3:
        soup = BeautifulSoup(str(winer_list[2]))
    contract_link = soup.find('a')
    contract_url = contract_link.get('href')
    contract_url = 'https://zakupki.gov.ru' + contract_url
    contract_page = requests.get(contract_url, headers = headers)
    contract_page_text = contract_page.content.decode('utf-8')
    soup = BeautifulSoup(contract_page_text)
    # Find winer info
    info = soup.find_all("tbody", {'class': "tableBlock__body"})[0]
    info_dict = {"inn": "","name": "", "phone": "", "address": ""}
    count_headers = len(soup.find_all("th", {"class": "tableBlock__col tableBlock__col_header"}))
    for i in range(len(info.find_all("td"))):
        if i == 0:
            info_dict["name"] = info.find_all("td")[i].text.split('\n')[1]
        if i == 2:
            info_dict["address"] = info.find_all("td")[i].text.strip()
        if i == count_headers - 2:
            info_dict["phone"] = info.find_all("td")[i].text.strip().split('\n')[0]
            info_dict["email"] = info.find_all("td")[i].text.strip().split('\n')[1]
        
    # Find winer inn
    spans = soup.find_all('span')
    ind_cur = -1
    for span in spans:
        if span.text == 'ИНН:': 
            ind_cur = spans.index(span) + 1
            break
    info_dict["inn"] = spans[ind_cur].text
    
    q.put(info_dict)
    return
tqdm.pandas()

In [51]:
def start_download(purch):
    out = purch
    q = queue.Queue()
    scraping = threading.Thread(target=get_winer_info_vie_purch, args=(out,q,))
    scraping.start()
    scraping.join()
    while threading.active_count() > 20:
        time.sleep(0.01)
    return q.get()

In [52]:
df = pd.DataFrame(columns=valid_df.columns)

In [53]:
inns = list(info_df.index)

In [54]:
for i in range(valid_df.shape[0]):
    if valid_df.iloc[i].winer in inns:
        df = df.append(valid_df.iloc[i])
        inns.remove(valid_df.iloc[i].winer)
        continue

In [56]:
df['info'] = df['purch'].progress_apply(start_download)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 541/541 [08:30<00:00,  1.06it/s]


In [57]:
info_df = info_df.reset_index()

In [58]:
df

,Закупки по,Реестровый номер закупки,"Способ определения поставщика (подрядчика, исполнителя), подрядной организации (размещения закупки)",Наименование закупки,Предмет электронного аукциона (только для ПП РФ 615),Идентификационный код закупки,Номер лота,Наименование лота,Начальная (максимальная) цена контракта,Валюта,...,Дата обновления,Этап закупки,Особенности размещения заказа\ Преимущества (44-ФЗ),Дата начала подачи заявок,Дата окончания подачи заявок,Дата проведения электронного аукциона,purch,winer,Год обновления,info
21042,44-ФЗ,№0356200002019000017,Электронный аукцион,Поставка лекарственных препаратов для медицинс...,NaN,'192590229047359020100100190130000000',NaN,NaN,1292250.00,RUB,...,2019-03-01,Определение поставщика завершено,"Участникам, заявки или окончательные предложен...",01.02.2019,28.02.2019,NaN,0356200002019000017,7726311464,2019,"{'inn': '7726311464', 'name': ' ..."
21117,44-ФЗ,№0373200041519000081,Открытый конкурс в электронной форме,Поставка в 2019 году кресел-колясок с ручным п...,NaN,'192770182732077020100100920010000323',NaN,NaN,9163000.00,RUB,...,2019-03-01,Определение поставщика завершено,NaN,01.02.2019,25.02.2019,NaN,0373200041519000081,3327110751,2019,"{'inn': '3327110751', 'name': ' ..."
21130,44-ФЗ,№0306200011519000029,Электронный аукцион,Поставка стерильного белья,NaN,'192100104211810010100100070071395000',NaN,NaN,3269950.00,RUB,...,2019-03-01,Определение поставщика завершено,NaN,01.02.2019,18.02.2019,NaN,0306200011519000029,6147011500,2019,"{'inn': '6147011500', 'name': ' ..."
21154,44-ФЗ,№0891200000619000096,Электронный аукцион,Поставка изделий медицинского назначения (89-ЭА),NaN,'192753602440875360100100190013250000',NaN,NaN,5616000.00,RUB,...,2019-02-28,Определение поставщика завершено,"Участникам, заявки или окончательные предложен...",01.02.2019,18.02.2019,NaN,0891200000619000096,7802829934,2019,"{'inn': '7802829934', 'name': ' ..."
21222,44-ФЗ,№0341200008919000014,Электронный аукцион,Поставка консервированных овощей (зеленый горо...,NaN,'192440200518044090100100120110000244',NaN,NaN,78297.50,RUB,...,2019-02-28,Определение поставщика завершено,NaN,01.02.2019,20.02.2019,NaN,0341200008919000014,503800018503,2019,"{'inn': '503800018503', 'name': ' ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426225,44-ФЗ,№0320100014121000017,Электронный аукцион,Поставка расходных материалов для хроматографии,NaN,'211251100253025110100100050022229244',NaN,NaN,229322.84,RUB,...,2021-06-24,Определение поставщика завершено,"Участникам, заявки или окончательные предложен...",02.06.2021,21.06.2021,NaN,0320100014121000017,2111008224,2021,"{'inn': '2111008224', 'name': ' ..."
427123,44-ФЗ,№0301100003721000021,Электронный аукцион,Поставка продуктов питания (фрукты) (СОЛ Авиатор),NaN,'211027402374702740100100400010000244',NaN,NaN,1204971.40,RUB,...,2021-06-21,Определение поставщика завершено,"Участникам, заявки или окончательные предложен...",02.06.2021,10.06.2021,NaN,0301100003721000021,0269998410,2021,"{'inn': '0269998410', 'name': ' ..."
463979,44-ФЗ,№0357200001521000015,Электронный аукцион,"Поставка бакалейных товаров, масла подсолнечно...",NaN,'212601200260660120100100330010000244',NaN,NaN,34917.20,RUB,...,2021-07-15,Определение поставщика завершено,"Субъектам малого предпринимательства, социальн...",02.07.2021,12.07.2021,NaN,0357200001521000015,602718413269,2021,"{'inn': '602718413269', 'name': ' ..."
509154,44-ФЗ,№0387200029719000043,Запрос котировок в электронной форме,Поставка персональных видеорегистраторов и ком...,NaN,'192860304247886030100102180012640000',NaN,NaN,474890.62,RUB,...,2019-09-19,Определение поставщика завершено,NaN,02.09.2019,10.09.2019,NaN,0387200029719000043,9709019150,2019,"{'inn': '9709019150', 'name': ' ..."


In [59]:
info_df["phone_number"] = info_df.winer.apply(lambda x: df[df["winer"] == x]["info"].iloc[0]["phone"])
info_df["address"] = info_df.winer.apply(lambda x: df[df["winer"] == x]["info"].iloc[0]["address"])
info_df["email"] = info_df.winer.apply(lambda x: df[df["winer"] == x]["info"].iloc[0]["email"] if "email" in df[df["winer"] == x]["info"].iloc[0] else "")
info_df["company_name"] = info_df.winer.apply(lambda x: df[df["winer"] == x]["info"].iloc[0]["name"])

In [60]:
info_df

,winer,2019 кол-во,2020 кол-во,2021 кол-во,2019 сумма,2020 сумма,2021 сумма,phone_number,address,email,company_name
0,0224013045,46,54,46,1.978899e+07,4.207305e+07,2.134629e+07,+7 (347) 293 02 93,"452414, РФ, РБ, Иглинский р-н, с.Карамалы, Фер...",biskom14@mail.ru,Общество с ограниченной ответс...
1,0269998410,4,8,10,7.493098e+05,3.165213e+06,7.507196e+06,8-919-6148500,"452754, РЕСП БАШКОРТОСТАН 02, Г ТУЙМАЗЫ, УЛ КО...",Frut1989@mail.ru,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТС...
2,0273041550,11,15,17,6.173826e+05,1.242729e+06,1.192723e+06,8-347-2647566,"450112, Российская Федерация, РЕСП БАШКОРТОСТ...",technika-om@mail.ru,ЧАСТНОЕ ОБРАЗОВАТЕЛЬНОЕ УЧРЕЖД...
3,0274110679,31,52,57,1.663205e+09,2.146573e+10,1.560091e+09,+7 (495) 970-00-30,"141701, область Московская город Долгопрудный ...",ookiryushina@pharm...,"АО ""Фармстандарт"""
4,0275900969,7,8,12,7.099961e+06,2.308415e+07,4.037719e+07,+7 (937) 305 48 83,"450077, республика Башкортостан, город Уфа, ул...",mirkator15@yandex.ru,Общество с ограниченной ответс...
...,...,...,...,...,...,...,...,...,...,...,...
536,9715229846,1,9,7,1.751351e+06,5.176178e+06,6.570980e+06,+7 (495) 544-27-35,"127576, Г МОСКВА 77, УЛ ИЛИМСКАЯ, ДОМ 2, ОФИС 37",benefis.tender@mai...,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТС...
537,9715322330,6,10,14,1.516988e+06,8.566969e+06,1.239015e+07,7-903-1733022,"127224, Г МОСКВА, УЛ СЕВЕРОДВИНСКАЯ, ДОМ 21, П...",info@enzowin.ru,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТС...
538,9715338098,1,9,22,2.313520e+06,2.561772e+07,3.081867e+07,7-495-5324503,"127576, Г МОСКВА 77, УЛ АБРАМЦЕВСКАЯ, ДОМ 8А, ...",analemma@list.ru,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТС...
539,9725006486,4,6,6,1.932473e+06,4.081942e+06,1.603439e+07,74957962964,"115114, Г МОСКВА, УЛ ДЕРБЕНЕВСКАЯ, ДОМ 11, ЭТА...",isida.info@yandex.ru,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТС...


In [61]:
info_df.to_excel("../data/inn_info.xlsx", index=False)